In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time

In [2]:
url= "https://learningsuite.byu.edu/.w60q/student/top/prioritizer"
r = requests.get(url)
text = r.text
soup= BeautifulSoup(r.text)


In [3]:
#trs = soup.find_all('tr', {'class':'h-10 bg-base'}{'data':'h-10 bg-base'})
trs = soup.find_all("tr", class_="h-10 bg-base")
trs


[]

In [9]:
%pip install CanvasScraper
%pip install canvasapi
%pip install pathvalidate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#!bin/python3
import argparse
import os
import re

from pathvalidate import sanitize_filename
from canvasapi import Canvas
from canvasapi.course import Course
from canvasapi.exceptions import Unauthorized, ResourceDoesNotExist, Forbidden
from canvasapi.file import File
from canvasapi.module import Module, ModuleItem


def extract_files(text):
    text_search = re.findall("/files/(\\d+)", text, re.IGNORECASE)
    groups = set(text_search)
    return groups

def get_course_files(course):
    modules = course.get_modules()

    files_downloaded = set() # Track downloaded files for this course to avoid duplicates

    for module in modules:
        module: Module = module
        module_items = module.get_module_items()
        for item in module_items:
            item: ModuleItem = item
            
            try:
                path = f"{output}/" \
                    f"{sanitize_filename(course.name)}/" \
                    f"{sanitize_filename(module.name)}/"
            except Exception as e:
                print(e)
                continue
            if not os.path.exists(path):
                os.makedirs(path)

            item_type = item.type
            print(f"{course.name} - "
                  f"{module.name} - "
                  f"{item.title} ({item_type})")

            if item_type == "File":
                file = canvas.get_file(item.content_id)
                files_downloaded.add(item.content_id)
                file.download(path + sanitize_filename(file.filename))
            elif item_type == "Page":
                page = course.get_page(item.page_url)
                with open(path + sanitize_filename(item.title) + ".html", "w", encoding="utf-8") as f:
                    f.write(page.body or "")
                files = extract_files(page.body or "")
                for file_id in files:
                    if file_id in files_downloaded:
                        continue
                    try:
                        file = course.get_file(file_id)
                        files_downloaded.add(file_id)
                        file.download(path + sanitize_filename(file.filename))
                    except ResourceDoesNotExist:
                        pass
            elif item_type == "ExternalUrl":
                url = item.external_url
                with open(path + sanitize_filename(item.title) + ".url", "w") as f:
                    f.write("[InternetShortcut]\n")
                    f.write("URL=" + url)
            elif item_type == "Assignment":
                assignment = course.get_assignment(item.content_id)
                with open(path + sanitize_filename(item.title) + ".html", "w", encoding="utf-8") as f:
                    f.write(assignment.description or "")
                files = extract_files(assignment.description or "")
                for file_id in files:
                    if file_id in files_downloaded:
                        continue
                    try:
                        file = course.get_file(file_id)
                        files_downloaded.add(file_id)
                        file.download(path + sanitize_filename(file.filename))
                    except ResourceDoesNotExist:
                        pass
                    except Unauthorized:
                        pass
                    except Forbidden:
                        pass

    try:
        files = course.get_files()
        for file in files:
            file: File = file
            if not file.id in files_downloaded:
                print(f"{course.name} - {file.filename}")
                path = f"{output}/{sanitize_filename(course.name)}/" \
                    f"{sanitize_filename(file.filename)}"
                file.download(path)
    except Unauthorized:
        pass
    except Forbidden:
        pass


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Download all assignments from Canvas")
    parser.add_argument("url", help="https://byu.instructure.com")
    parser.add_argument("token", help="7407~VB3XeGUnteMkUXPkmC7rETc2K3838uW7x7GVv8Mywx2nKhWDA4XTCVz7VmyWGmAF")
    parser.add_argument("output", help="Path to the output folder, e.g. output/")
    parser.add_argument("courses", help="Comma-separated course IDs or 'all'", nargs="?", const="all")
    args = parser.parse_args()

    # Handle args
    output = args.output.rstrip("/") + "/"

    if args.courses is None:
        args.courses = "all"
        print("No courses specified. Scraping all courses.")

    canvas = Canvas(args.url, args.token)

    courses = [] # courses to scrape

    # Select courses to scrape, default to all
    if args.courses != "all":
        courses = []
        ids = args.courses.split(",")
        for id in ids:
            courses.append(canvas.get_course( int(id) ))
    else:
        courses = canvas.get_courses()

    # Perform scrape
    for course in courses:
        course: Course = course
        get_course_files(course)

usage: ipykernel_launcher.py [-h] url token output [courses]
ipykernel_launcher.py: error: the following arguments are required: url, token, output


SystemExit: 2

In [36]:
import requests

with open('canvasapi.txt', 'r') as f:
    TOKEN = f.read()

# API_URL = "https://byu.instructure.com/api/v1/courses"

# headers = {"Authorization": f"Bearer {TOKEN}"}
# response = requests.get(API_URL, headers=headers)

# print(response.json()) 


#CS111 27227
#math 29215
COURSE_ID = 27227

API_URL = "https://byu.instructure.com/api/v1/courses/{COURSE_ID}/assignment_groups"

# BASE_URL = "https://byu.instructure.com"

# # Verify base API access
# response = requests.get(f"{BASE_URL}/api/v1/courses", headers={"Authorization": f"Bearer {TOKEN}"})
# print("Courses Response:", response.status_code, response.json())

# # Check if course exists
# response = requests.get(f"{BASE_URL}/api/v1/courses/{COURSE_ID}", headers={"Authorization": f"Bearer {TOKEN}"})
# print("Course Details Response:", response.status_code, response.json())

# # Check assignments
# API_URL = f"{BASE_URL}/api/v1/courses/{COURSE_ID}/assignments"
# response = requests.get(API_URL, headers={"Authorization": f"Bearer {TOKEN}"})
# print("Assignments Response:", response.status_code, response.json())

headers = {
    "Authorization": f"Bearer {TOKEN}"
}

# response = requests.get(API_URL, headers=headers)

# print(response.json())


# if response.status_code == 200:
#     assignments = response.json()
#     for assignment in assignments:
#         name = assignment["name"]
#         due_date = assignment.get("due_at", "No due date")
#         print(f"Assignment: {name} | Due Date: {due_date}")
# else:
#     print(f"Error: {response.status_code}, {response.text}")

# API_URL = f"https://byu.instructure.com/api/v1/courses/{COURSE_ID}/students/submissions"
response = requests.get(API_URL, headers=headers)
print(response.status_code, response.json())


404 {'errors': [{'message': 'The specified resource does not exist.'}]}


In [ ]:
url= "https://learningsuite.byu.edu/.w60q/student/top/prioritizer"

In [14]:
cookies = {'session_id': '004bd8fb-f8a1-4041-98c5-6fb131180971|EXISTING'}  # ✅ Dictionary (correct)

# Get the calendar page
response = requests.get(url, cookies=cookies)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract calendar data (modify this based on actual HTML structure)
events = []
for event in soup.find_all("tr", class_="h-10 bg-base"):  # Adjust this class
    date = event.find("time", datetime_="2025-01-14T06:59:00.000Z").text.strip()
    title = event.find("span", class_="overflow-hidden sm:-64 w-48 sm:w-auto pl-3").text.strip()
    course = event.find("span", class_="sm:hidden text-2xs text-info").text.strip()
    events.append([date, title, course])
print(events)
# Convert to DataFrame & Save as CSV
# df = pd.DataFrame(events, columns=["Date", "Assignment", "Course"])
# df.to_csv("learning_suite_calendar.csv", index=False)

# print("Calendar exported successfully!")

[]


In [15]:
%pip install selenium


Note: you may need to restart the kernel to use updated packages.
